# Visualization
Roboquant Notebooks come out of the box with support for several types of charts. They will help to better understand what is going on during a run and evaluate the performance of the underlying strategy. 

Under the hood roboquant leverages the excellent [Apache ECharts](https://echarts.apache.org/en/index.html)  visualization library to render the charts in Notebooks. ECharts was selected because:

- it has a very feature rich set of charts
- it can handle large data sets
- easy to add some interactive behavior like filtering and selection
- Apache opensource project and license

By default roboquant tries to detect the environment it is running in, either Jupyter-Notebook or Jupyter-Lab. However this detection is not 100% accurate. But you can always invoke *Ouput.lab()* or *Output.notebook()* so roboquant knows the environment and will generate the correct output. 

In [24]:
%use roboquant
Welcome()

roboquant (version 0.8-SNAPSHOT) 
 home: /home/peter/.roboquant 
 os: Linux 5.13.19-200.fc34.x86_64 
 jvm: OpenJDK 64-Bit Server VM 11.0.12 
 memory: 7946 MB 
 env: LAB

# Price charts

A feed represents a stream of events you can use in your strategies. There are several type of feeds supported, like the one for Avro files used here. The file using in this notebook contains 5 years of S&P 500 daily prices (OHLCV).

The Avro file format allows to store, access and compress data efficiently and works great for historic prices used in back-testing. This feed implementation also has low memory consumption since it only loads data when required. If you want to find out more about Avro files, check out their [documentation pages](https://avro.apache.org/docs/current/index.html).


Roboquant also supports plain CSV files as well as several external sources like Yahoo Finance. More detailed examples can be found in the *Feeds Sample Notebook* and the roboquant documentation.

In [2]:
val feed = AvroFeed.sp500()

// Show the first 10 assets in this feed
feed.assets.take(10).summary()

Assets
├── STOCK SLB 
├── STOCK KLAC 
├── STOCK MDT 
├── STOCK ECL 
├── STOCK AVGO 
├── STOCK JNPR 
├── STOCK UPS 
├── STOCK ADI 
├── STOCK WMB 
└── STOCK DUK 


In [3]:
val apple = feed.find("AAPL")
PriceChart(feed, apple)

## PriceBar Chart
Roboqant isn't a technical analysis tool where you draw some support lines in a candlestick chart. But that doesn't mean candlesticks cannot come in handy. For example to validate if the data in a feed is not corrupted or has some other strange anomalies.

In this example we'll find Apple stock (symbol name is "AAPL") and plot it as a candle-stick chart.

In [4]:
PriceBarChart(feed, apple)

# Single Run
We run a simple strategy over the feed we just created to get some data to use for the charts. For this notebook we use the EMA crossover strategy with its default settings and capture the AccountSummary metrics. 

In [5]:
val strategy = EMACrossover()
val roboquant =  Roboquant(strategy, AccountSummary())
roboquant.run(feed)

2013-02-08 - 2018-02-07 | 100% |████████████████████████████████████████| Roboquant-0 | run=1 | phase=MAIN |


## Asset charts
Lets see what assets are in our portfolio at the end of the run. We can plot only the assets (pie chart) or plot the assets per asset-class (sunburst chart). Optionally we can also say if the cash positions in the account should be included in the chart.

In [6]:
val account = roboquant.broker.account

AssetAllocationChart(account, includeCash = false, includeAssetClass=false)

In [7]:
AssetAllocationChart(account, includeCash = true, includeAssetClass=true)

In [8]:
// You can ofcourse also just print a summary
account.portfolio.summary()

Portfolio
├──          asset │  curr │        qty │      entry │       spot │        p&l │
├──       STOCK:MA │   USD │        133 │     150.57 │     168.73 │    2415.22 │
├──     STOCK:ROST │   USD │        341 │      58.86 │      77.99 │    6523.74 │
├──      STOCK:OKE │   USD │        375 │      53.35 │      56.58 │    1210.01 │
├──    STOCK:GOOGL │   USD │         20 │     957.21 │    1055.41 │    1964.08 │
├──      STOCK:AIG │   USD │        155 │      60.70 │      60.47 │     -35.75 │
├──      STOCK:CNC │   USD │        204 │      95.77 │     104.38 │    1757.32 │
├──     STOCK:ANSS │   USD │        157 │     127.24 │     155.03 │    4363.48 │
├──       STOCK:IP │   USD │        316 │      57.01 │      59.25 │     708.80 │
├──      STOCK:VAR │   USD │        165 │     123.12 │     116.92 │   -1023.50 │
├──       STOCK:KR │   USD │        885 │      21.72 │      28.90 │    6352.79 │
├──      STOCK:EMN │   USD │        235 │      85.24 │      98.98 │    3230.04 │
├──      STOCK:COP

## Trades
Another important aspect is to see which trades has been executed as part of the back-test and how did they perform.

In [9]:
TradeChart(account.trades, skipBuy = true)

You can also plot other aspects of a trade, like the amount or fees.  

In [10]:
TradeChart(account.trades, aspect = "amount")

In [11]:
PriceBarChart(feed, apple, account.trades)

##  Orders 
We can also view the orders that were created as part of the run. This time we don't plot the orders of all assets, but only of AMD.

In [12]:
val orders = account.orders.filter { it.asset.symbol == "AMD" }
OrderChart(orders)

## Metrics

Besides charts for account related information, you can also plot the metrics that where captured. We used the AccountSummary metric that keeps track of how our account performs during the back-test. We use the account value (= cash + positions), for this sample

In [13]:
val logger = roboquant.logger

// What metrics are available
logger.getMetricNames().summary("Metrics Names")

Metrics Names
├── account.orders
├── account.orders.open
├── account.trades
├── account.portfolio.assets
├── account.cash.currencies
├── account.cash.total
├── account.buyingPower
├── account.value
└── account.change


In [14]:
val data = logger.getMetric("account.value")
MetricChart(data)

## Metric Box Chart
If you want to inspect how volatile the performance of a strategy is, the metric box chart comes in handy. It provides insights into how a metric is distributed over a period of time. If provides the following info per box:

- The minimum
- low percentile (default is 25.0)
- mid percentile (default is 50.0, aka median)
- high percentile (default is 75.0)
- the maximum 

So in this case it shows these statsistics of the **account value change** based on the monthly aggregate of daily values.

In [ ]:
val data = logger.getMetric("account.change")
MetricBoxChart(data)

In [ ]:
MetricBoxChart(data, period="year", lowPercentile = 10.0, highPercentile = 90.0)

## Metric histogram
If we want to see the distribution of values over the metric, we can use the MetricHistogram chart. Besides the metric data that needs to be plot, we can also specify the number of bins. On the x-axis the upperbound of each bin are displayed and on the y-axis how many oberservations fit in that bin.

In [ ]:
MetricHistogram(data, 15)

# Multi Run back-test
A good way to back-test your strategy is to run it multiple times over different time-frames. This section shows some charts that come in handy to better understand the results of those type of back-tests. 

## Walk-forward
A commonly used approach in back-testing is the so called walk-forward. The historic data is split in a number of equal periods and then the strategy is run for each period seperately. This shows how the strategy would have performed during these different periods and possible different regimes.

In the following code we split the data in periods of 1 year and for each period run it. We then plot again the account value as observed during each run.

In [ ]:
roboquant.reset()

feed.split(Period.ofYears(1)).forEach { 
    roboquant.run(feed, it)   
}

val data = roboquant.logger.getMetric("account.value")
MetricChart(data)

## Random sampling
Another great way to back-test your strategy is to run it for a fixed duration at many random starting points in time. This gives you a good insight what could be a good and bad outcome of your strategy. The following piece of code runs the strategy 25 times, each time for the duration of 250 trading days. Feel free to change the number of runs from 25 to a higher value like 50 or 100. It is kept small so that also people with older or less powerfull computers don't have to wait too long.

In this case we plot the *account value* (= cash + positions) again and you can see there are times we do well but there are also periods when we loose money.

In [19]:
roboquant.reset()
roboquant.logger.showProgress = false

repeat(25) {
    val timeFrame = feed.sample(250)
    roboquant.run(feed, timeFrame) 
}

val data = roboquant.logger.getMetric("account.value")
MetricChart(data, useTime=false)

If you want to see the same data, but this time plotted in time, you can use the same MetricChart class and provide the *useTime=true* parameter. This shows in which periods the strategy is performing well and in which periods it is struggeling. It is similar to the walk-forward approach, but provides at each moment in time multiple samples.

In [20]:
MetricChart(data, useTime=true)

# Other

## Calender chart
When evaluating the performance of a strategy, it is key to find out when the strategy was performing good and bad. A calendar charts allows you to plot a metric value per day, so it is easy to find out what days to further investigate. The slider at top of the chart allows for filtering only those days that are of interest.

In the example below we plot the unrealized profit and loss to see when the strategy performed good (green) and when bad (red). It also demonstrates how to run it only for a limited timeframe.

In [21]:
val strategy = EMACrossover()
val roboquant =  Roboquant(strategy, AccountSummary())
roboquant.run(feed, TimeFrame.fromYears(2014, 2016))

2014-01-01 - 2017-01-01 | 100% |████████████████████████████████████████| Roboquant-1 | run=1 | phase=MAIN |


In [22]:
val data = roboquant.logger.getMetric("account.change")
val chart = CalendarChart(data)

// You can also save charts to a file
// chart.toHTMLFile("/tmp/calendar_result.html")
chart

## Displaying multiple charts

By using the *chart.render()* method you can display multiple charts in the output section of a single cell. The following code iterates over the assets in a feed of historic data and renders a candlestick chart for each one of them. It also demonstrates the excellent performance of the chart library, since every charts contains 20 years of daily data.

In [23]:
val assets = feed.assets.random(3)
for (asset in assets) PriceBarChart(feed, asset).render()